In [12]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances 

In [15]:
#Col1 = User Col2 = Jokes Col3 = Ratings
df = np.load('DATAROW.npy')
ratings = np.copy(df[:,2])
m = ratings.shape[0]
ratings = np.reshape(ratings,[m,1])
#Get mean values
clean_ratings = ratings[~np.isnan(ratings).any(axis=1)] 
mean = np.mean(clean_ratings)

#Replace nan with mean value
ratings[np.isnan(ratings)] = mean
user = np.reshape(df[:,0] , [m,1])
jokes = np.reshape(df[:,1], [m,1])
data = np.concatenate((user,jokes,ratings), axis = 1)

In [20]:
#Split into train test 
train_data, test_data = train_test_split(data ,test_size = 0.99)

In [21]:
print(train_data.shape)
print(test_data.shape)

(36710, 3)
(3634290, 3)


In [44]:
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0

MemoryError: 

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0

In [25]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [26]:
user_prediction = predict(train_data, user_correlation, type='user')
item_prediction = predict(train_data, item_correlation, type='item')


In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [36]:
# RMSE on the train data
print('User-based CF MSE: ' + str(rmse(user_prediction, train_data)))
print('Item-based CF MSE: ' + str(rmse(item_prediction, train_data)))

User-based CF MSE: 5004.97095976063
Item-based CF MSE: 558.5162920362097


In [47]:
print(user_prediction.shape)


(36710, 3)


In [48]:
print(train_data.shape)

(36710, 3)
